# Xarray Access CMIP5 Data 


In this notebook we demonstrate how to access CMIP5 data locally and remotely using Xarray.

* What is Xarray
* Remote vs. direct filesystem access
* Build a multi-file dataset from an OpenDAP server
* File Variables and Attributes

This example uses Coupled Model Intercomparison Project (CMIP5) collections. For more information, please visit [data catalogue](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f3525_9322_8600_7716) and [terms of use](https://cmip.llnl.gov/cmip5/terms.html).
   
---------

- Authors: NCI Virtual Research Environment Team
- Keywords: CMIP, Xarray, NetCDF
- Create Date: 2019-Jun; Update Date: 2020-Apr

### Prerequisite

To run this notebook on Gadi/VDI, or on your local computer. The following modules are needed:

* Xarray
* netcdf-python

You also need to be a member of the following data project to access the data or through [NCI's data service](http://dapds00.nci.org.au/thredds/catalog.html):
* rr3

You can request to join the project through [NCI's user account management system](https://my.nci.org.au). 

### Xarray

Xarray builds upon and extends the strengths of pandas and numpy. Numpy provides the structure and core for working with multi-dimensional arrays while pandas integrates its indexing and dataframe type capabilities. Xarray is actively developed by the climate science community and a useful tool for analysis. For more information on the developments being undertaken (along with other related projects) see the Pangeo community: https://pangeo.io/
 
We will use xarray to open the CMIP5 file defined below. Opening a file with xarray creates an xarray.Dataset. A 'Dataset' is a collection of multiple variables. A DataArray on the other hand is a single multi-dimensional variable and the coordinates.
 
xarray always loads netCDF data 'lazily', this means that data can be manipulated, sliced and subset without loading array values into memory. Data is loaded into memory when the load( ) command is applied or when a computaiton is performed on the data.
 
xarray is designed for use with multidimensional datasets and is particularly useful for climate data on multidimensional grids with dimensions such as lat, lon, depth and time

#### Import the xarray and netCDF modules

In [1]:
import xarray as xr
import netCDF4 as nc
%matplotlib inline

### Remote vs. direct filesystem access

In this example, we will use a file from the CMIP5 Australian Published data collection, specifically the monthly historical tasmax data:

    /g/data/rr3/publications/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/mon/atmos/Amon/r1i1p1/v20120727/tasmax/tasmax_Amon_ACCESS1-0_historical_r1i1p1_185001-200512.nc
    

and we are going to compare direct vs. remote access. Timings (using the `%%time` magic function) will also be shown to help illustrate when it can be useful to conduct analysis on the filesystem.

#### Local path on /g/data

In [2]:
path = '/g/data/rr3/publications/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/mon/atmos/\
Amon/r1i1p1/v20130325/tasmax/tasmax_Amon_ACCESS1-3_historical_r1i1p1_185001-200512.nc'

#### OPeNDAP Data URL

For more information on where to find OPeNDAP URL's, see:
<a href="https://nbviewer.jupyter.org/github/nci-training/readthedoc_NCI_data_training/blob/master/docs/_notebook/TDS/tds_OPeNDAP_cmip5.ipynb">THREDDS Data Server: Data Access</a>


In [3]:
url = 'http://dapds00.nci.org.au/thredds/dodsC/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/mon/atmos/\
Amon/r1i1p1/v20130325/tasmax/tasmax_Amon_ACCESS1-3_historical_r1i1p1_185001-200512.nc'

#### Open the file, comparing the time on the local filesystem and remote url

In [4]:
%%time
f1 = xr.open_dataset(path, decode_times=False)

CPU times: user 240 µs, sys: 162 ms, total: 162 ms
Wall time: 1.25 s


In [ ]:
%%time
f2 = xr.open_dataset(url)

The difference in times is due to the remote URL access. Data access response time is quick using Xarray's `open_dataset()` function because of the lazy loading of data. But if force the data to load into memory:

In [6]:
%%time
f1 = xr.open_dataset(path, decode_times=False)
tasmax = f1.tasmax
tasmax.load()

CPU times: user 358 ms, sys: 155 ms, total: 513 ms
Wall time: 511 ms


<xarray.DataArray 'tasmax' (time: 1872, lat: 145, lon: 192)>
array([[[242.22649, 242.22649, 242.22649, ..., 242.22028, 242.22028,
         242.22028],
        [243.91931, 243.87326, 243.82875, ..., 244.05038, 244.00957,
         243.96425],
        [245.13287, 245.08662, 245.0183 , ..., 245.27603, 245.23296,
         245.18419],
        ...,
        [242.64438, 242.74596, 242.86943, ..., 242.36124, 242.45232,
         242.54388],
        [241.69199, 241.74554, 241.79771, ..., 241.67577, 241.68051,
         241.69809],
        [240.87485, 240.87485, 240.87485, ..., 240.87485, 240.87485,
         240.87485]],

       [[235.21791, 235.21791, 235.21791, ..., 235.2146 , 235.2146 ,
         235.2146 ],
        [237.052  , 237.029  , 237.00833, ..., 237.17393, 237.13855,
         237.09853],
        [237.41914, 237.29343, 237.1765 , ..., 237.80144, 237.6658 ,
         237.53809],
        ...,
        [240.60106, 240.53203, 240.48082, ..., 240.71024, 240.71312,
         240.65256],
        [238.93102, 238.94348, 238.95735, ..., 238.84721, 238.8793 ,
         238.91093],
        [236.43845, 236.43845, 236.43845, ..., 236.43845, 236.43845,
         236.43845]],

       [[223.82248, 223.82248, 223.82248, ..., 223.81923, 223.81923,
         223.81923],
        [228.1175 , 228.05615, 228.00998, ..., 228.28644, 228.22849,
         228.17545],
        [229.21747, 229.08847, 228.9618 , ..., 229.61053, 229.47765,
         229.34256],
        ...,
        [243.81131, 243.88744, 243.97946, ..., 243.57245, 243.6507 ,
         243.72934],
        [243.39563, 243.40419, 243.41534, ..., 243.32387, 243.34251,
         243.37648],
        [242.78352, 242.78352, 242.78352, ..., 242.78352, 242.78352,
         242.78352]],

       ...,

       [[222.91371, 222.91371, 222.91371, ..., 222.90314, 222.90314,
         222.90314],
        [227.95497, 227.90009, 227.84071, ..., 228.13326, 228.07675,
         228.0165 ],
        [230.00499, 229.81673, 229.62323, ..., 230.51114, 230.35124,
         230.18073],
        ...,
        [257.4335 , 257.61444, 257.79364, ..., 256.95648, 257.11493,
         257.26663],
        [257.3382 , 257.4279 , 257.54178, ..., 257.04166, 257.12665,
         257.2381 ],
        [257.07758, 257.07758, 257.07758, ..., 257.07758, 257.07758,
         257.07758]],

       [[240.10124, 240.10124, 240.10124, ..., 240.09166, 240.09166,
         240.09166],
        [241.85622, 241.80452, 241.75043, ..., 241.99152, 241.93628,
         241.88435],
        [242.79108, 242.68707, 242.58368, ..., 243.11673, 243.01666,
         242.9077 ],
        ...,
        [260.98044, 261.00912, 261.07172, ..., 260.78656, 260.87418,
         260.9435 ],
        [260.00974, 260.02213, 260.0323 , ..., 259.96994, 259.99246,
         260.0092 ],
        [258.00223, 258.00223, 258.00223, ..., 258.00223, 258.00223,
         258.00223]],

       [[247.96088, 247.96088, 247.96088, ..., 247.93704, 247.93704,
         247.93704],
        [248.94565, 248.89906, 248.85555, ..., 249.08572, 249.03673,
         248.98915],
        [249.93575, 249.80705, 249.679  , ..., 250.31178, 250.1902 ,
         250.0654 ],
        ...,
        [251.01753, 251.10835, 251.17427, ..., 250.68848, 250.80872,
         250.9194 ],
        [249.8625 , 249.90404, 249.93315, ..., 249.72313, 249.77002,
         249.82123],
        [248.79779, 248.79779, 248.79779, ..., 248.79779, 248.79779,
         248.79779]]], dtype=float32)
Coordinates:
  * time     (time) float64 6.753e+05 6.754e+05 ... 7.323e+05 7.323e+05
  * lat      (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
    height   float64 1.5
Attributes:
    standard_name:     air_temperature
    long_name:         Daily Maximum Near-Surface Air Temperature
    comment:           monthly mean of the daily-maximum near-surface air tem...
    units:             K
    cell_methods:      time: maximum within days time: mean over days
    cell_measures: 

In [ ]:
%%time
f2 = xr.open_dataset(url)
tasmax = f2.tasmax
tasmax.load()

<div class="alert alert-info">
One big advantage of working directly on the filesystem is that data access is much faster. For modest subsets, the difference is quite small but as you work with larger data, remote access can become much slower or even exceed NCI's THREDDS Data Server memory limits. </div>

### Build a multi-file dataset from an OpenDAP server

In [11]:
base_url = 'http://dapds00.nci.org.au/thredds/catalog/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/\
historical/day/ocean/day/r1i1p1/latest/tos/tos_day_ACCESS1-3_historical_r1i1p1_'
files = [f'{base_url}{year}0101_{year+9}1231.nc' for year in range(1850, 1990, 10)]
files

['http://dapds00.nci.org.au/thredds/catalog/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/day/ocean/day/r1i1p1/latest/tos/tos_day_ACCESS1-3_historical_r1i1p1_18500101_18591231.nc',
 'http://dapds00.nci.org.au/thredds/catalog/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/day/ocean/day/r1i1p1/latest/tos/tos_day_ACCESS1-3_historical_r1i1p1_18600101_18691231.nc',
 'http://dapds00.nci.org.au/thredds/catalog/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/day/ocean/day/r1i1p1/latest/tos/tos_day_ACCESS1-3_historical_r1i1p1_18700101_18791231.nc',
 'http://dapds00.nci.org.au/thredds/catalog/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/day/ocean/day/r1i1p1/latest/tos/tos_day_ACCESS1-3_historical_r1i1p1_18800101_18891231.nc',
 'http://dapds00.nci.org.au/thredds/catalog/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/day/ocean/day/r1i1p1/latest/tos/tos_day_ACCESS1-3_historical_r1i1p1_18900101_18991231.nc',
 'http://dapds00.nci.org.au/thredds/catalog/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-

<div class="alert alert-warning">
Notes: Formatter string literals is used in constructing multiple files above. This uitlity is only available in python 3.6 and above. If you run this example using current VDI setting, this won't work. Please replace the above code using traditional string formatting method, something like this would work: </div>

In [ ]:
base_url = 'http://dapds00.nci.org.au/thredds/catalog/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/\
historical/day/ocean/day/r1i1p1/latest/tos/tos_day_ACCESS1-3_historical_r1i1p1_'
files = []
for year in range(1850, 1990, 10):
    file = print('{0}{1}0101_{2}1231.nc'.format(base_url, year, year+9))
    files.append(file)

We can now the entire ensemble of files on the remote server as a single xarray Dataset using `Xarray.mfdataset( )`!

In [12]:
f3 = xr.open_mfdataset(files,decode_times=False)
f3

OSError: [Errno -68] NetCDF: I/O failure: b'http://dapds00.nci.org.au/thredds/catalog/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/day/ocean/day/r1i1p1/latest/tos/tos_day_ACCESS1-3_historical_r1i1p1_18500101_18591231.nc'

### File variables and attributes

With xarray, you can easily view the dataset variables and attributes contained in the file by printing the loaded metadata

In [ ]:
f1 = xr.open_dataset(path, decode_times=False)
print(f1)

### Dataset and DataArray

In the above we have loaded the Dataset and you can see the multiple variables included in the file. If we look at a specific variable, like tasmax, we will get an xarray.DataArray with its coordinates.

In [ ]:
f1 = xr.open_dataset(path, decode_times=False)
print(f1.tasmax)

#### Print an attribute
The attributes of a variable can be easily accessed using the `.<attribute>` command. So if we want to print the units of tasmax we could go:

In [ ]:
f1.tasmax.units

### Summary

In this example, we show how to do access CMIP5 datasets on Gadi and through URL. The same approach applies to other data collections. Please visit [Xarray](http://xarray.pydata.org/en/stable/) for more information.